In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("NLP_Example").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/03/16 12:05:52 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/03/16 12:05:53 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [2]:
from pyspark.ml.feature import HashingTF, IDF, Tokenizer

In [16]:
sen_df = spark.createDataFrame(
    [
        (0, "Hi I heard about Spark"),
        (1, "i wish java could use case classes"),
        (2, "Logistic,regression,models,are,neat"),
    ],
    ["label", "sentence"],
)
sen_df.show()

+-----+--------------------+
|label|            sentence|
+-----+--------------------+
|    0|Hi I heard about ...|
|    1|i wish java could...|
|    2|Logistic,regressi...|
+-----+--------------------+



In [17]:
tokenizer = Tokenizer(inputCol="sentence", outputCol="words")

In [18]:
words_data = tokenizer.transform(sen_df)

In [19]:
words_data.show(truncate=False)

+-----+-----------------------------------+------------------------------------------+
|label|sentence                           |words                                     |
+-----+-----------------------------------+------------------------------------------+
|0    |Hi I heard about Spark             |[hi, i, heard, about, spark]              |
|1    |i wish java could use case classes |[i, wish, java, could, use, case, classes]|
|2    |Logistic,regression,models,are,neat|[logistic,regression,models,are,neat]     |
+-----+-----------------------------------+------------------------------------------+



In [20]:
hashing_tf = HashingTF(inputCol="words", outputCol="rawFeatures")

In [21]:
featurized_data = hashing_tf.transform(words_data)

In [22]:
idf = IDF(inputCol="rawFeatures", outputCol="features")

In [23]:
idf_model = idf.fit(featurized_data)

In [24]:
rescaled_data = idf_model.transform(featurized_data)

In [25]:
rescaled_data.show()

24/03/16 12:13:08 WARN DAGScheduler: Broadcasting large task binary with size 4.0 MiB
24/03/16 12:13:08 WARN DAGScheduler: Broadcasting large task binary with size 4.0 MiB


+-----+--------------------+--------------------+--------------------+--------------------+
|label|            sentence|               words|         rawFeatures|            features|
+-----+--------------------+--------------------+--------------------+--------------------+
|    0|Hi I heard about ...|[hi, i, heard, ab...|(262144,[18700,19...|(262144,[18700,19...|
|    1|i wish java could...|[i, wish, java, c...|(262144,[19036,20...|(262144,[19036,20...|
|    2|Logistic,regressi...|[logistic,regress...|(262144,[11534],[...|(262144,[11534],[...|
+-----+--------------------+--------------------+--------------------+--------------------+



24/03/16 12:13:08 WARN DAGScheduler: Broadcasting large task binary with size 4.0 MiB


In [27]:
rescaled_data.select("label", "features").show(truncate=False)

24/03/16 12:13:26 WARN DAGScheduler: Broadcasting large task binary with size 4.0 MiB
24/03/16 12:13:26 WARN DAGScheduler: Broadcasting large task binary with size 4.0 MiB


+-----+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|label|features                                                                                                                                                                                      |
+-----+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|0    |(262144,[18700,19036,33808,66273,173558],[0.6931471805599453,0.28768207245178085,0.6931471805599453,0.6931471805599453,0.6931471805599453])                                                   |
|1    |(262144,[19036,20719,55551,58672,98717,109547,192310],[0.28768207245178085,0.6931471805599453,0.6931471805599453,0.6931471805599453,0.6931471805599453,0.6931471805599453,0.6931471805599453])|
|2   

24/03/16 12:13:26 WARN DAGScheduler: Broadcasting large task binary with size 4.0 MiB


In [28]:
from pyspark.ml.feature import CountVectorizer

In [29]:
# Input data: Each row is a bag of words with a ID.
df = spark.createDataFrame(
    [(0, "a b c".split(" ")), (1, "a b b c a".split(" "))], ["id", "words"]
)

In [30]:
df.show()

+---+---------------+
| id|          words|
+---+---------------+
|  0|      [a, b, c]|
|  1|[a, b, b, c, a]|
+---+---------------+



In [31]:
cv = CountVectorizer(inputCol="words", outputCol="features", vocabSize=3, minDF=2.0)

In [32]:
model = cv.fit(df)

In [33]:
result = model.transform(df)

In [34]:
result.show(truncate=False)

+---+---------------+-------------------------+
|id |words          |features                 |
+---+---------------+-------------------------+
|0  |[a, b, c]      |(3,[0,1,2],[1.0,1.0,1.0])|
|1  |[a, b, b, c, a]|(3,[0,1,2],[2.0,2.0,1.0])|
+---+---------------+-------------------------+

